# Part 3 - 3D Geometry viewing

It is also possible to create a 3D representation of CSG geometry. The model is converted into a pixelated geometry which can resemble the style of geometry seen in the minecraft computer game.

This is not an actual representation of the CSG geometry but more like a sample of the types of cells with 3D coordinates. CAD geometry is covered later and this gives us a method of seeing the 3D geometry more accurately.

3D pixelated geometry can still be useful for exploring the model; particularly if the resolution is high enough. However, this can be a memory intensive task.

This python notebook allows users to construct CSG geometry and view a 3D representation.

This first code block recreates the simple reactor geometry seen in Part 2, but also assigns materials to each cell.

In [ ]:
import openmc
import os

mats = openmc.Materials()

copper = openmc.Material(name='Copper')
copper.set_density('g/cm3', 8.5)
copper.add_element('Cu', 1.0)  # Note, percent_type does not have to be specified as material is 100% copper
mats.append(copper)

eurofer = openmc.Material(name='steel')
eurofer.set_density('g/cm3', 7.75)
eurofer.add_element('Fe', 99.9, percent_type='wo')
eurofer.add_element('C', 0.1, percent_type='wo')

mats.append(eurofer)

breeder_material = openmc.Material(name='breeder_material')
breeder_material.set_density('g/cm3', 9.1)
breeder_material.add_element('Pb', 84.2, percent_type='ao')
breeder_material.add_element('Li', 15.8, percent_type='ao')
mats.append(breeder_material)

mats.export_to_xml()

# define all the surfaces
central_sol_surface = openmc.ZCylinder(r=100)
central_shield_outer_surface = openmc.ZCylinder(r=110, boundary_type='vacuum')
vessel_inner = openmc.Sphere(r=500, boundary_type='vacuum')
first_wall_outer_surface = openmc.Sphere(r=510)
breeder_blanket_outer_surface = openmc.Sphere(r=610)


# define the cells
central_sol_region = -central_sol_surface & -breeder_blanket_outer_surface
central_sol_cell = openmc.Cell(region=central_sol_region)
central_sol_cell.fill = copper

central_shield_region = +central_sol_surface & -central_shield_outer_surface & -breeder_blanket_outer_surface
central_shield_cell = openmc.Cell(region=central_shield_region)
central_shield_cell.fill = eurofer

first_wall_region = -first_wall_outer_surface & +vessel_inner & +central_shield_outer_surface
first_wall_cell = openmc.Cell(region=first_wall_region)
first_wall_cell.fill = eurofer

breeder_blanket_region = +first_wall_outer_surface & -breeder_blanket_outer_surface & +central_shield_outer_surface
breeder_blanket_cell = openmc.Cell(region=breeder_blanket_region)
breeder_blanket_cell.fill = breeder_material

universe = openmc.Universe(cells=[central_sol_cell,central_shield_cell,first_wall_cell, breeder_blanket_cell])

geom = openmc.Geometry(universe)

geom.export_to_xml()

# A blank settings.xml is exported to allow the openmc plotter to work
sett = openmc.Settings()
sett.export_to_xml()

The next code block runs OpenMC in geometry mode and produces a plot_3d_tokamak.h5 file. Particles are not transported through the geometry in this mode - it simply samples the geometry on the grid and makes a h5 output file with the results.

In [ ]:
# makes the 3d "cube" style geometry
vox_plot = openmc.Plot()
vox_plot.type = 'voxel'
vox_plot.width = (1500., 1500., 1500.)
vox_plot.pixels = (200, 200, 200)
vox_plot.filename = 'plot_3d_tokamak'
vox_plot.color_by = 'material'
# vox_plot.colors = {copper: 'blue'}  # materials can be coloured using this command
plots = openmc.Plots([vox_plot])
plots.export_to_xml()

openmc.plot_geometry()

This code block runs a small conversion script that is provided with OpenMC. The script reads in the 3D array from the h5 file and converts it to a vti file that can be viewed using compatable software such as Paraview or Visit.

In [ ]:
# this converts the h5 file to a vti
os.system('openmc-voxel-to-vtk plot_3d_tokamak.h5 -o plot_3d_tokamak.vti')

Run this final command to generate a download link to the 3D vtk file.

In [ ]:
from IPython.display import FileLink
display(FileLink('plot_3d_tokamak.vti'))

Click on the link to download the vtk file onto your base computer and open it with a VTK file reader such as Paraview or Visit.

Paraview can be downloaded here: https://www.paraview.org/download/. Visit can be downloaded here: https://wci.llnl.gov/simulation/computer-codes/visit/downloads.

**Learning Outcomes for Part 3**

- CSG models can be converted to vti files and viewed in 3D with tools such as Paraview or Visit.